## Documentation

To read more about the common options in Elasticsearch, checkout the docs [here](https://www.elastic.co/guide/en/elasticsearch/reference/current/common-options.html#date-math).

![common_options_docs](../images/common_options_docs.png)

## Connect to ElasticSearch

In [1]:
from pprint import pprint
from elasticsearch import Elasticsearch

es = Elasticsearch('http://localhost:9200')
client_info = es.info()
print('Connected to Elasticsearch!')
pprint(client_info.body)

Connected to Elasticsearch!
{'cluster_name': 'es-docker-cluster',
 'cluster_uuid': '68vsKryIR7Ss49bLh7mz5Q',
 'name': 'es01',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2025-12-16T10:09:08.849001802Z',
             'build_flavor': 'default',
             'build_hash': 'd8972a71dbbd64ff17f2f4dba9ca2c3fe09fb100',
             'build_snapshot': False,
             'build_type': 'docker',
             'lucene_version': '10.3.2',
             'minimum_index_compatibility_version': '8.0.0',
             'minimum_wire_compatibility_version': '8.19.0',
             'number': '9.2.3'}}


## 1. Human-readable output

In [2]:
response = es.cluster.stats(human=True)
pprint(response["nodes"]["jvm"])

{'max_uptime': '6.6m',
 'max_uptime_in_millis': 399828,
 'mem': {'heap_max': '1gb',
         'heap_max_in_bytes': 1073741824,
         'heap_used': '313.3mb',
         'heap_used_in_bytes': 328541792},
 'threads': 143,
 'versions': [{'bundled_jdk': True,
               'count': 1,
               'using_bundled_jdk': True,
               'version': '25.0.1',
               'vm_name': 'OpenJDK 64-Bit Server VM',
               'vm_vendor': 'Oracle Corporation',
               'vm_version': '25.0.1+8-27'}]}


In [3]:
response = es.cluster.stats(human=False)
pprint(response["nodes"]["jvm"])

{'max_uptime_in_millis': 411062,
 'mem': {'heap_max_in_bytes': 1073741824, 'heap_used_in_bytes': 366290528},
 'threads': 144,
 'versions': [{'bundled_jdk': True,
               'count': 1,
               'using_bundled_jdk': True,
               'version': '25.0.1',
               'vm_name': 'OpenJDK 64-Bit Server VM',
               'vm_vendor': 'Oracle Corporation',
               'vm_version': '25.0.1+8-27'}]}


## 2. Date math

In [4]:
es.indices.delete(index='my_index', ignore_unavailable=True)
es.indices.create(index='my_index')

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'my_index'})

In [5]:
import json

operations = []
index_name = 'my_index'
dummy_data = json.load(open("../data/dummy_data.json"))
for document in dummy_data:
    operations.append({'index': {'_index': index_name}})
    operations.append(document)

es.bulk(operations=operations)

ObjectApiResponse({'errors': False, 'took': 200, 'items': [{'index': {'_index': 'my_index', '_id': 'LZzkc5sB53kGhVbdMdJa', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'my_index', '_id': 'Lpzkc5sB53kGhVbdMdJa', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'my_index', '_id': 'L5zkc5sB53kGhVbdMdJa', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 2, '_primary_term': 1, 'status': 201}}]})

In [6]:
response = es.search(
    index=index_name,
    body={
        "query": {
            "range": {
                "created_on": {
                    "gte": "2024-09-22||+1d/d",  # 2024-09-23
                    "lte": "now/d"  # 2024-11-16
                }
            }
        }
    }
)
hits = response['hits']['hits']
print(f"Found {len(hits)} documents")

Found 2 documents


In [7]:
response = es.search(
    index=index_name,
    body={
        "query": {
            "range": {
                "created_on": {
                    "gte": "2024-09-22||+1M/d",  # 2024-09-22 + 1 month
                    "lte": "now/d"
                }
            }
        }
    }
)
hits = response['hits']['hits']
print(f"Found {len(hits)} documents")

Found 0 documents


## 3. Response filtering

### 3.1 Inclusive filtering

In [8]:
response = es.search(
    index=index_name,
    body={
        "query": {
            "match_all": {}
        }
    },
)
pprint(response.body)

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [{'_id': 'LZzkc5sB53kGhVbdMdJa',
                    '_index': 'my_index',
                    '_score': 1.0,
                    '_source': {'created_on': '2024-09-22',
                                'text': 'This is the first sample document '
                                        'text.',
                                'title': 'Sample Title 1'}},
                   {'_id': 'Lpzkc5sB53kGhVbdMdJa',
                    '_index': 'my_index',
                    '_score': 1.0,
                    '_source': {'created_on': '2024-09-24',
                                'text': 'Here is another example of a '
                                        'document.',
                                'title': 'Sample Title 2'}},
                   {'_id': 'L5zkc5sB53kGhVbdMdJa',
                    '_index': 'my_index',
                    '_score': 1.0,
                    '_source': {'created_on': '2024-0

In [9]:
response = es.search(
    index=index_name,
    body={
        "query": {
            "match_all": {}
        }
    },
    filter_path="hits.hits._id,hits.hits._source"  # Keep only _id and _source fields
)
pprint(response.body)

{'hits': {'hits': [{'_id': 'LZzkc5sB53kGhVbdMdJa',
                    '_source': {'created_on': '2024-09-22',
                                'text': 'This is the first sample document '
                                        'text.',
                                'title': 'Sample Title 1'}},
                   {'_id': 'Lpzkc5sB53kGhVbdMdJa',
                    '_source': {'created_on': '2024-09-24',
                                'text': 'Here is another example of a '
                                        'document.',
                                'title': 'Sample Title 2'}},
                   {'_id': 'L5zkc5sB53kGhVbdMdJa',
                    '_source': {'created_on': '2024-09-24',
                                'text': 'The content of the third document '
                                        'goes here.',
                                'title': 'Sample Title 3'}}]}}


### 3.2 Exclusive filtering

In [10]:
response = es.search(
    index=index_name,
    body={
        "query": {
            "match_all": {}
        }
    },
    filter_path="-hits"  # Remove the hits key
)
pprint(response.body)

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'timed_out': False,
 'took': 1}


### 3.3. Combined filtering

In [11]:
response = es.search(
    index=index_name,
    body={
        "query": {
            "match_all": {}
        }
    },
    filter_path="hits.hits._id,-hits.hits._score"
)
pprint(response.body)

{'hits': {'hits': [{'_id': 'LZzkc5sB53kGhVbdMdJa'},
                   {'_id': 'Lpzkc5sB53kGhVbdMdJa'},
                   {'_id': 'L5zkc5sB53kGhVbdMdJa'}]}}


## 4. Flat settings

In [12]:
response = es.indices.get_settings(
    index=index_name,
)
pprint(response.body)

{'my_index': {'settings': {'index': {'creation_date': '1767175888112',
                                     'number_of_replicas': '1',
                                     'number_of_shards': '1',
                                     'provided_name': 'my_index',
                                     'routing': {'allocation': {'include': {'_tier_preference': 'data_content'}}},
                                     'uuid': 'qubT6i-ETe2AkScY_8_ngQ',
                                     'version': {'created': '9039003'}}}}}


In [13]:
response = es.indices.get_settings(
    index=index_name,
    flat_settings=True,
)
pprint(response.body)

{'my_index': {'settings': {'index.creation_date': '1767175888112',
                           'index.number_of_replicas': '1',
                           'index.number_of_shards': '1',
                           'index.provided_name': 'my_index',
                           'index.routing.allocation.include._tier_preference': 'data_content',
                           'index.uuid': 'qubT6i-ETe2AkScY_8_ngQ',
                           'index.version.created': '9039003'}}}
